                              Brain Tumor Detection

We collected data from three different sources, One having all the images with differentitaion in one place, others are in no and yes folders seperately.



1.   Lets read the images using csv , differentiate them into two sub folders.
2.   After differentitation, we need to dump into the no and yes classes
     seperately to train the model
3.   We got anohter source of images mentionly named no and yes tumors.so
     we can directly copy these into same folder that we have already.
3.   Mount the folders yes and no from drive, then train the model accordingly.



Importing Depandencies

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [3]:
import glob

In [4]:
import tensorflow as tf
from tensorflow.keras import layers,models
import os
from PIL import Image

In [5]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator #Data Augumentation
from tensorflow.keras.callbacks import ModelCheckpoint

In [6]:
from tensorflow.keras import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D,GlobalAveragePooling2D, Input, Dropout, LeakyReLU,UpSampling2D, concatenate
from keras.models import load_model, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import DenseNet121

In [7]:
import shutil

Data Importing

In [9]:
!git clone "https://github.com/aswindam/Brain_tumor.git"

Cloning into 'Brain_tumor'...
remote: Enumerating objects: 26512, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 26512 (delta 0), reused 255 (delta 0), pack-reused 26257
Receiving objects: 100% (26512/26512), 186.38 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (635/635), done.
Updating files: 100% (26823/26823), done.


In [10]:
!mkdir Brain_tumor_dataset

In [11]:
!cp -r Brain_tumor/have_tumor Brain_tumor_dataset/
!cp -r Brain_tumor/have_tumor_1/* Brain_tumor_dataset/have_tumor/
!cp -r Brain_tumor/no_tumor Brain_tumor_dataset/

Preprocessing

In [12]:
# Set the paths to the train and test folders
train_folder = 'Brain_tumor_dataset'
test_folder = 'Brain_tumor/Testing'

In [13]:
data_gen = ImageDataGenerator(rescale=1./255)

# Load the images from the train and test folders as arrays
train_data = data_gen.flow_from_directory(train_folder, target_size=(224, 224), batch_size=32, shuffle=False, class_mode='binary')
test_data = data_gen.flow_from_directory(test_folder, target_size=(224, 224), batch_size=32, shuffle=False, class_mode='binary')

Found 26429 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [14]:
def cnnmodelrelu():

    # Define the checkpoint filepath
    checkpoint_filepath_relu = 'best_model_checkpoint_relu.h5'

    # Create the ModelCheckpoint callback
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath_relu,
                                          monitor='val_accuracy',
                                          save_best_only=True,
                                          mode='max',
                                          verbose=1)

    print('CNN with relu activation')
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



    cnn = model.fit(
        train_data,
        steps_per_epoch=train_data.samples // train_data.batch_size,
        epochs=20,
        validation_data=test_data,  # Provide validation data
        validation_steps=test_data.samples // test_data.batch_size,
        callbacks=[checkpoint_callback]
    )

    best_model = load_model(checkpoint_filepath_relu)
    best_accuracy = max(cnn.history['val_accuracy'])
    print(f"Best validation accuracy: {best_accuracy} (Epoch {cnn.history['val_accuracy'].index(best_accuracy) + 1})")


cnnmodelrelu()

CNN with relu activation
Epoch 1/20
825/825 [==============================] - ETA: 0s - loss: 0.6008 - accuracy: 0.7334
Epoch 1: val_accuracy improved from -inf to 0.74219, saving model to best_model_checkpoint_relu.h5
825/825 [==============================] - 64s 64ms/step - loss: 0.6008 - accuracy: 0.7334 - val_loss: 0.6029 - val_accuracy: 0.7422
Epoch 2/20
825/825 [==============================] - ETA: 0s - loss: 0.4914 - accuracy: 0.7782
Epoch 2: val_accuracy improved from 0.74219 to 0.74479, saving model to best_model_checkpoint_relu.h5
825/825 [==============================] - 51s 61ms/step - loss: 0.4914 - accuracy: 0.7782 - val_loss: 0.6025 - val_accuracy: 0.7448
Epoch 3/20
825/825 [==============================] - ETA: 0s - loss: 0.5100 - accuracy: 0.7826
Epoch 3: val_accuracy improved from 0.74479 to 0.74740, saving model to best_model_checkpoint_relu.h5
825/825 [==============================] - 54s 66ms/step - loss: 0.5100 - accuracy: 0.7826 - val_loss: 0.5809 - val_ac

CNN - Activation FN Leaky Relu

In [15]:
def cnnmodelleakyr():


    # Define the checkpoint filepath
    checkpoint_filepath_leaky = 'best_model_checkpoint_leakyrelu.h5'

    # Create the ModelCheckpoint callback
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath_leaky,
                                          monitor='val_accuracy',
                                          save_best_only=True,
                                          mode='max',
                                          verbose=1)

    print('CNN with Leaky Relu...')
    # Create the model
    model1 = Sequential()
    model1.add(Conv2D(32, (3, 3), activation=LeakyReLU(alpha=0.2), input_shape=(224, 224, 3)))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(64, (3, 3), activation=LeakyReLU(alpha=0.2)))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(128, (3, 3), activation=LeakyReLU(alpha=0.2)))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(256, (3, 3), activation=LeakyReLU(alpha=0.2)))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Flatten())
    model1.add(Dense(128, activation='relu'))
    model1.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model with the checkpoint callback
    leak = model1.fit(
        train_data,
        steps_per_epoch=train_data.samples // train_data.batch_size,
        epochs=20,
        validation_data=test_data,  # Provide validation data
        validation_steps=test_data.samples // test_data.batch_size,
        callbacks=[checkpoint_callback]  # Add the checkpoint callback to the training

    )

    # Load the best model based on validation accuracy
    best_model = load_model(checkpoint_filepath_leaky)
    best_accuracy = max(leak.history['val_accuracy'])
    print(f"Best validation accuracy: {best_accuracy} (Epoch {leak.history['val_accuracy'].index(best_accuracy) + 1})")


cnnmodelleakyr()

CNN with Leaky Relu...
Epoch 1/20
825/825 [==============================] - ETA: 0s - loss: 0.6678 - accuracy: 0.6920
Epoch 1: val_accuracy improved from -inf to 0.74479, saving model to best_model_checkpoint_leakyrelu.h5
825/825 [==============================] - 58s 67ms/step - loss: 0.6678 - accuracy: 0.6920 - val_loss: 0.6777 - val_accuracy: 0.7448
Epoch 2/20
825/825 [==============================] - ETA: 0s - loss: 0.5219 - accuracy: 0.7564
Epoch 2: val_accuracy improved from 0.74479 to 0.74740, saving model to best_model_checkpoint_leakyrelu.h5
825/825 [==============================] - 54s 65ms/step - loss: 0.5219 - accuracy: 0.7564 - val_loss: 0.6088 - val_accuracy: 0.7474
Epoch 3/20
825/825 [==============================] - ETA: 0s - loss: 0.4761 - accuracy: 0.7747
Epoch 3: val_accuracy did not improve from 0.74740
825/825 [==============================] - 54s 65ms/step - loss: 0.4761 - accuracy: 0.7747 - val_loss: 0.6559 - val_accuracy: 0.7292
Epoch 4/20
825/825 [========

Trying VGG16

In [16]:
def vgg():

    # Define the checkpoint filepath
    checkpoint_filepath_vgg = 'best_model_checkpoint_vgg.h5'

    # Create the ModelCheckpoint callback
    checkpoint_callback_vgg = ModelCheckpoint(filepath=checkpoint_filepath_vgg,
                                          monitor='val_accuracy',
                                          save_best_only=True,
                                          mode='max',
                                          verbose=1)
    print('Classification using VGG')

    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vggmodel = Sequential()
    vggmodel.add(base_model)
    vggmodel.add(Flatten())
    vggmodel.add(Dense(4096, activation=LeakyReLU(alpha=0.2)))
    vggmodel.add(Dense(4096, activation=LeakyReLU(alpha=0.2)))
    vggmodel.add(Dense(1, activation='sigmoid'))
    vggmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model with the checkpoint callback
    vggm = vggmodel.fit(
        train_data,
        steps_per_epoch=train_data.samples // train_data.batch_size,
        epochs=10,
        validation_data=test_data,  # Provide validation data
        validation_steps=test_data.samples // test_data.batch_size,
        callbacks=[checkpoint_callback_vgg]  # Add the checkpoint callback to the training

    )

    # Load the best model based on validation accuracy
    best_model = load_model(checkpoint_filepath_vgg)
    best_accuracy = max(vggm.history['val_accuracy'])
    print(f"Best validation accuracy: {best_accuracy} (Epoch {vggm.history['val_accuracy'].index(best_accuracy) + 1})")

vgg()

Classification using VGG
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
825/825 [==============================] - ETA: 0s - loss: 9.8550 - accuracy: 0.5222
Epoch 1: val_accuracy improved from -inf to 0.75260, saving model to best_model_checkpoint_vgg.h5
825/825 [==============================] - 361s 416ms/step - loss: 9.8550 - accuracy: 0.5222 - val_loss: 0.6117 - val_accuracy: 0.7526
Epoch 2/10
825/825 [==============================] - ETA: 0s - loss: 0.7008 - accuracy: 0.5094
Epoch 2: val_accuracy did not improve from 0.75260
825/825 [==============================] - 325s 394ms/step - loss: 0.7008 - accuracy: 0.5094 - val_loss: 0.6329 - val_accuracy: 0.7526
Epoch 3/10
825/825 [==============================] - ETA: 0s - loss: 0.7032 - accuracy: 0.5202
Epoch 3: val_accuracy did not improve from 0.75260
825/825 [==============================] - 324s 393ms/step - loss: 0.7032 - accuracy: 0.5202 - val_loss: 0.6305 - val_accuracy: 0.7526
Epoch 4/10
825/82

ResNet

In [17]:
def resnetm():

    # Define the checkpoint filepath
    checkpoint_filepath_resnet = 'best_model_checkpoint_resnet.h5'

    # Create the ModelCheckpoint callback
    checkpoint_callback_resnet = ModelCheckpoint(filepath=checkpoint_filepath_resnet,
                                                 monitor='val_accuracy',
                                                 save_best_only=True,
                                                 mode='max',
                                                 verbose=1)

    print('Classification using ResNet')

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    resnet_model = GlobalAveragePooling2D()(base_model.output)
    resnet_model = Dense(256, activation='relu')(resnet_model)
    predictions = Dense(1, activation='sigmoid')(resnet_model)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model with the checkpoint callback
    history = model.fit(
        train_data,
        epochs=20,
        batch_size=20,
        validation_data=test_data,
        callbacks=[checkpoint_callback_resnet]
    )

    # Load the best model based on validation accuracy
    best_model = load_model(checkpoint_filepath_resnet)
    best_accuracy = max(history.history['val_accuracy'])
    print(
        f"Best validation accuracy: {best_accuracy} (Epoch {history.history['val_accuracy'].index(best_accuracy) + 1})")

resnetm()

Classification using ResNet
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/20
826/826 [==============================] - ETA: 0s - loss: 0.5019 - accuracy: 0.7971
Epoch 1: val_accuracy improved from -inf to 0.30457, saving model to best_model_checkpoint_resnet.h5
826/826 [==============================] - 318s 335ms/step - loss: 0.5019 - accuracy: 0.7971 - val_loss: 0.8459 - val_accuracy: 0.3046
Epoch 2/20
826/826 [==============================] - ETA: 0s - loss: 0.1760 - accuracy: 0.9492
Epoch 2: val_accuracy improved from 0.30457 to 0.67766, saving model to best_model_checkpoint_resnet.h5
826/826 [==============================] - 272s 330ms/step - loss: 0.1760 - accuracy: 0.9492 - val_loss: 0.9066 - val_accuracy: 0.6777
Epoch 3/20
826/826 [==============================] - ETA: 0s - loss: 0.1500 - accuracy: 0.9569
Epoch 3: val_accuracy improved from 0.67766 to 0.73350, saving model to best_model_checkpoint_resnet.h5
826/826 [==============================]

Inception

In [18]:
def inceptionv3():

    # Define the checkpoint filepath
    checkpoint_filepath_inception = 'best_model_checkpoint_inception.h5'

    # Create the ModelCheckpoint callback
    checkpoint_callback_inception = ModelCheckpoint(filepath=checkpoint_filepath_inception,
                                                    monitor='val_accuracy',
                                                    save_best_only=True,
                                                    mode='max',
                                                    verbose=1)

    print('Classification using InceptionV3')

    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    inception_model = GlobalAveragePooling2D()(base_model.output)
    inception_model = Dense(256, activation='relu')(inception_model)
    predictions = Dense(1, activation='sigmoid')(inception_model)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model with the checkpoint callback
    history = model.fit(
        train_data,
        epochs=20,
        batch_size=20,
        validation_data=test_data,
        callbacks=[checkpoint_callback_inception]
    )

    # Load the best model based on validation accuracy
    best_model = load_model(checkpoint_filepath_inception)
    best_accuracy = max(history.history['val_accuracy'])
    print(
        f"Best validation accuracy: {best_accuracy} (Epoch {history.history['val_accuracy'].index(best_accuracy) + 1})")

inceptionv3()


Classification using InceptionV3
87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/20
826/826 [==============================] - ETA: 0s - loss: 0.6852 - accuracy: 0.5306
Epoch 1: val_accuracy improved from -inf to 0.36041, saving model to best_model_checkpoint_inception.h5
826/826 [==============================] - 243s 241ms/step - loss: 0.6852 - accuracy: 0.5306 - val_loss: 0.6967 - val_accuracy: 0.3604
Epoch 2/20
826/826 [==============================] - ETA: 0s - loss: 0.6836 - accuracy: 0.5352
Epoch 2: val_accuracy improved from 0.36041 to 0.73350, saving model to best_model_checkpoint_inception.h5
826/826 [==============================] - 190s 231ms/step - loss: 0.6836 - accuracy: 0.5352 - val_loss: 0.6360 - val_accuracy: 0.7335
Epoch 3/20
826/826 [==============================] - ETA: 0s - loss: 0.5478 - accuracy: 0.7140
Epoch 3: val_accuracy did not improve from 0.73350
826/826 [==============================] - 189s 229ms/step - loss: 0.5478 - accura

Dense Net

In [19]:
def densenet121():

    # Define the checkpoint filepath
    checkpoint_filepath_densenet = 'best_model_checkpoint_densenet.h5'

    # Create the ModelCheckpoint callback
    checkpoint_callback_densenet = ModelCheckpoint(filepath=checkpoint_filepath_densenet,
                                                   monitor='val_accuracy',
                                                   save_best_only=True,
                                                   mode='max',
                                                   verbose=1)

    print('Classification using DenseNet121')

    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    densenet_model = GlobalAveragePooling2D()(base_model.output)
    densenet_model = Dense(256, activation='relu')(densenet_model)
    predictions = Dense(1, activation='sigmoid')(densenet_model)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model with the checkpoint callback
    history = model.fit(
        train_data,
        epochs=20,
        batch_size=20,
        validation_data=test_data,
        callbacks=[checkpoint_callback_densenet]
    )

    # Load the best model based on validation accuracy
    best_model = load_model(checkpoint_filepath_densenet)
    best_accuracy = max(history.history['val_accuracy'])
    print(
        f"Best validation accuracy: {best_accuracy} (Epoch {history.history['val_accuracy'].index(best_accuracy) + 1})")

densenet121()


Classification using DenseNet121
29084464/29084464 [==============================] - 0s 0us/step
Epoch 1/20
826/826 [==============================] - ETA: 0s - loss: 0.4378 - accuracy: 0.7893
Epoch 1: val_accuracy improved from -inf to 0.72081, saving model to best_model_checkpoint_densenet.h5
826/826 [==============================] - 372s 361ms/step - loss: 0.4378 - accuracy: 0.7893 - val_loss: 1.0990 - val_accuracy: 0.7208
Epoch 2/20
826/826 [==============================] - ETA: 0s - loss: 0.1776 - accuracy: 0.9449
Epoch 2: val_accuracy did not improve from 0.72081
826/826 [==============================] - 287s 348ms/step - loss: 0.1776 - accuracy: 0.9449 - val_loss: 0.8579 - val_accuracy: 0.6320
Epoch 3/20
826/826 [==============================] - ETA: 0s - loss: 0.1283 - accuracy: 0.9654
Epoch 3: val_accuracy improved from 0.72081 to 0.73604, saving model to best_model_checkpoint_densenet.h5
826/826 [==============================] - 289s 349ms/step - loss: 0.1283 - accuracy

Testing using image downloaded from net..

In [73]:
image_path1 = '/kaggle/input/datase/test 1.jpg'
image1 = Image.open(image_path1)
image1 = image1.resize((224, 224))  # Resize the image to match the input size of your model
image1 = np.array(image1)  # Convert image to numpy array
image1 = image1 / 255.0  # Normalize the image

image_path2 = '/kaggle/input/testprede/test 2.jpg'
image2 = Image.open(image_path2)
image2 = image2.resize((224, 224))  # Resize the image to match the input size of your model
image2 = np.array(image2)  # Convert image to numpy array
image2 = image2 / 255.0  # Normalize the image

image_path3 = '/kaggle/input/test3img/test 3.jpg'
image3 = Image.open(image_path3)
image3 = image3.resize((224, 224))  # Resize the image to match the input size of your model
image3 = np.array(image3)  # Convert image to numpy array
image3 = image3 / 255.0  # Normalize the image


# Load the model from the HDF5 file
model_path = '/kaggle/working/best_model_checkpoint_relu.h5'
model = tf.keras.models.load_model(model_path)

In [74]:
input_data1 = np.expand_dims(image1, axis=0)  # Adding an extra dimension for batch
predictions1 = model.predict(input_data1)

# Interpret the results
class_names = ['no_tumor', 'have_tumor']  # Define your class names
predicted_class_index1 = np.argmax(predictions1)
predicted_class1 = class_names[predicted_class_index1]
confidence1 = predictions1[0][predicted_class_index1]

print('Prediction for image 1:', predicted_class1)
print('Confidence that model did predict is :', confidence1*100)

1/1 [==============================] - 0s 77ms/step
Prediction for image 1: no_tumor
Confidence that model did predict is : 21.08895182609558


In [75]:
# Making predictions for image 2
input_data2 = np.expand_dims(image2, axis=0)  # Adding an extra dimension for batch
predictions2 = model.predict(input_data2)

# Interpret the results
predicted_class_index2 = np.argmax(predictions2)
predicted_class2 = class_names[predicted_class_index2]
confidence2 = predictions2[0][predicted_class_index2]

print('Prediction for image 2 ---', predicted_class2)
print('Confidence that model did predict is :', confidence2 * 100)

1/1 [==============================] - 0s 20ms/step
Prediction for image 2 --- no_tumor
Confidence that model did predict is : 99.99998807907104


In [76]:
# Making predictions for image 2
input_data3 = np.expand_dims(image3, axis=0)  # Adding an extra dimension for batch
predictions3 = model.predict(input_data3)

# Interpret the results
predicted_class_index3 = np.argmax(predictions3)
predicted_class3 = class_names[predicted_class_index3]
confidence3 = predictions3[0][predicted_class_index3]

print('Prediction for image 3:', predicted_class3)
print('Confidence that model did predict is :', confidence3 * 100)


1/1 [==============================] - 0s 23ms/step
Prediction for image 3: no_tumor
Confidence that model did predict is : 0.16265135491266847
